In [33]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D,BatchNormalization,UpSampling1D,Conv1DTranspose,Dense,Flatten,Reshape,Attention, Add, Multiply, Activation,LeakyReLU
from tensorflow.keras.models import Sequential,Model
from mat4py import loadmat
from keras.layers import LSTM
from tensorflow.keras.optimizers import Adam,SGD
import scipy.io
from biosppy.signals import ecg
from scipy import signal
from math import log10, sqrt
from keras.layers import LSTM
from keras import backend as K
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import LSTM, Input, Dropout
from sklearn.linear_model import LinearRegression
from sklearn import datasets
from ecgdetectors import Detectors
from scipy.signal import find_peaks
from scipy.signal import butter, filtfilt
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
import random

In [34]:
def normalize_ecg(ecg_signal):
    """
    Normalize the ECG signal using min-max normalization
    """
    # Find the minimum and maximum values of the signal
    min_val = np.min(ecg_signal)
    max_val = np.max(ecg_signal)

    # Normalize the signal using min-max normalization
    normalized_ecg = (ecg_signal - min_val) / (max_val - min_val)

    return normalized_ecg

In [35]:
mat = scipy.io.loadmat('115m.mat')

In [36]:
aa=mat['val']
a=aa[0,0:]
a=normalize_ecg(a)
a.shape

(650000,)

In [37]:
# Original sampling rate and desired new sampling rate
original_sampling_rate =360    # Replace with your actual sampling rate
new_sampling_rate = 250  # Desired new sampling rate

# Calculate the new length of the resampled signal
original_length = len(a)
new_length = int(original_length * (new_sampling_rate / original_sampling_rate))

# Resample the signal
r_ecg = signal.resample(a, new_length)

In [38]:
out = ecg.christov_segmenter(signal=r_ecg, sampling_rate=250)
rpeaks = out['rpeaks']

In [39]:
segment_length = 600  # Fixed segment length of 600 data points
window_size_seconds = 2.4  # Time window of 2.4 seconds
segments = []


for i in range(0, len(rpeaks) - 3, 2):  # Step by 2 to get two R-peaks per segment
    rpeak1 = rpeaks[i]
    rpeak2 = rpeaks[i + 1]
    
    # Calculate the midpoint between the two R-peaks
    midpoint = (rpeak1 + rpeak2) // 2
    
    # Calculate the segment start and end indices for the specified window size
    window_size_samples = int(250 * window_size_seconds)
    segment_start = max(0, midpoint - window_size_samples // 2)
    segment_end = segment_start + segment_length
    
    # Check if segment_end goes beyond the signal length
    if segment_end > len(r_ecg):
        # Perform zero-padding if necessary
        segment = np.pad(r_ecg[segment_start:], (0, segment_end - len(r_ecg)))
    else:
        segment = r_ecg[segment_start:segment_end]
    
    segments.append(segment)

In [40]:
data=np.array(segments)
# data1=data.reshape((200,300))
data.shape

(785, 600)

In [41]:
Y_train, Y_test = train_test_split(data, test_size=0.2)
Y_test.shape
y=len(Y_test)

In [42]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

In [43]:
latent_dim = 1
encoder_inputs = keras.Input(shape=(600, 1))
x=Conv1D(16,13,padding='same')(encoder_inputs)
x=LeakyReLU(alpha=0.2)(x)
x=MaxPooling1D(2)(x)

x=Conv1D(32,7,padding='same')(x)
x=LeakyReLU(alpha=0.2)(x)
x=MaxPooling1D(2)(x)

x=Conv1D(64,7,padding='same')(x)
x=LeakyReLU(alpha=0.2)(x)
x=MaxPooling1D(2)(x)

x=Conv1D(128,7,padding='same')(x)
x=LeakyReLU(alpha=0.2)(x)

conv_shape=K.int_shape(x)
x=Flatten()(x)
z_mean = layers.Dense(latent_dim, name="z_mean")(x)
z_log_var = layers.Dense(latent_dim, name="z_log_var")(x)
z = Sampling()([z_mean, z_log_var])
encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 600, 1)]     0           []                               
                                                                                                  
 conv1d_14 (Conv1D)             (None, 600, 16)      224         ['input_5[0][0]']                
                                                                                                  
 leaky_re_lu_16 (LeakyReLU)     (None, 600, 16)      0           ['conv1d_14[0][0]']              
                                                                                                  
 max_pooling1d_6 (MaxPooling1D)  (None, 300, 16)     0           ['leaky_re_lu_16[0][0]']         
                                                                                            

In [44]:
latent_inputs = keras.Input(shape=(latent_dim,))
x=Dense(conv_shape[1]*conv_shape[2],activation='relu')(latent_inputs)
x=Reshape((conv_shape[1],conv_shape[2]))(x)
# decoder_inputs=encoder(inputs)[2]
x=Conv1DTranspose(128,7,padding='same')(x)
x=LeakyReLU(alpha=0.2)(x)

x=Conv1D(128, kernel_size=3, padding='same')(x)

x=Conv1DTranspose(64,7,padding='same')(x)
x=LeakyReLU(alpha=0.2)(x)
x=UpSampling1D(2)(x)

x=Conv1D(64, kernel_size=3, padding='same')(x)

x=Conv1DTranspose(32,7,padding='same')(x)
x=LeakyReLU(alpha=0.2)(x)
x=UpSampling1D(2)(x)

x=Conv1D(32, kernel_size=3, padding='same')(x)

x=Conv1DTranspose(16,13,padding='same')(x)
x=LeakyReLU(alpha=0.2)(x)
x=UpSampling1D(2)(x)
x=Flatten()(x)
decoder_outputs=Dense(600,activation='linear')(x)

# decoder_outputs=Reshape((2000,1))(decoder_outputs)
decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")
decoder.summary()

Model: "decoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_6 (InputLayer)        [(None, 1)]               0         
                                                                 
 dense_4 (Dense)             (None, 9600)              19200     
                                                                 
 reshape_2 (Reshape)         (None, 75, 128)           0         
                                                                 
 conv1d_transpose_8 (Conv1DT  (None, 75, 128)          114816    
 ranspose)                                                       
                                                                 
 leaky_re_lu_20 (LeakyReLU)  (None, 75, 128)           0         
                                                                 
 conv1d_18 (Conv1D)          (None, 75, 128)           49280     
                                                           

In [45]:
class VAE(keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super().__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.reduce_sum(keras.losses.mean_squared_error(data, reconstruction), axis=-1)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }

In [46]:
vae = VAE(encoder,decoder)
vae.compile(optimizer=keras.optimizers.Adam())
vae.fit(Y_train, epochs=50, batch_size=32)

Epoch 1/50
20/20 [==============================] - 11s 224ms/step - loss: 6.9278 - reconstruction_loss: 4.4056 - kl_loss: 0.0309
Epoch 2/50
20/20 [==============================] - 4s 200ms/step - loss: 0.3696 - reconstruction_loss: 0.2085 - kl_loss: 0.0097
Epoch 3/50
20/20 [==============================] - 5s 233ms/step - loss: 0.0449 - reconstruction_loss: 0.0388 - kl_loss: 4.4666e-04
Epoch 4/50
20/20 [==============================] - 7s 354ms/step - loss: 0.0241 - reconstruction_loss: 0.0237 - kl_loss: 2.8048e-05
Epoch 5/50
20/20 [==============================] - 7s 344ms/step - loss: 0.0233 - reconstruction_loss: 0.0223 - kl_loss: 5.9591e-06
Epoch 6/50
20/20 [==============================] - 7s 356ms/step - loss: 0.0134 - reconstruction_loss: 0.0219 - kl_loss: 8.0060e-07
Epoch 7/50
20/20 [==============================] - 7s 361ms/step - loss: 0.0142 - reconstruction_loss: 0.0222 - kl_loss: 3.0582e-07
Epoch 8/50
20/20 [==============================] - 7s 356ms/step - loss: 0.

In [47]:
z, _, _ = vae.encoder.predict(Y_test)
pred=vae.decoder.predict(z)
# # pred=np.array(pred)
#pred=vae.decoder.predict(vae.encoder.predict(X_test)[2])
pred.shape

(157, 600)

In [48]:
p1=0
s1=0;
for x in range(y):
    rec1=pred[x]
    xtrn1=Y_test[x]
    actual= xtrn1
    predicted= rec1
    sme= np.sum(np.square(actual - predicted))
    org = np.sum(np.square(actual))
    prd1= np.sqrt(sme/org)*100
    p1=p1+prd1;
    signal_power = np.sum(actual ** 2) / len(actual)
    noise_signal = actual-predicted
    noise_power = np.sum(noise_signal ** 2) / len(noise_signal)
    snr_db = 10 * np.log10(signal_power / noise_power)
    s1=s1+snr_db;

prd=p1/y;
snr_d=s1/y;
print("PRD:",prd) 
print("SNR:",snr_d)
    

PRD: 3.3947677934920235
SNR: 30.130893286153245
